In [ ]:
from trader_kin import Trader
import pandas as pd
from io import StringIO
import json
import re

from datamodel import OrderDepth, TradingState, Listing, Order

In [87]:
def read_submission_log(filename: str) -> tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    with open(filename, 'r') as f:
        log_content = f.read()

    sandbox, market = log_content.split("Sandbox logs:")[1].split("Activities log:")
    market, trade_history = market.split("Trade History:")

    market = pd.read_csv(StringIO(market.strip()), sep=';')
    trade_history = json.loads(trade_history.strip())
    trade_history = pd.DataFrame(trade_history)
    sandbox = [json.loads(s) for s in re.findall(r'\{.*?\}', sandbox, re.DOTALL)]
    sandbox = pd.DataFrame(sandbox)
    return market, trade_history, sandbox
    
market_data, trade_history, sandbox = read_submission_log('logs/empty_submission.log')

In [ ]:
trader = Trader()

class Backtester():
    def __init__(self, trader: Trader, market_data: pd.DataFrame, trade_history: pd.DataFrame, listings: dict[str, Listing], position_limits: dict[str, int]):
        self.trader = trader
        self.market_data = market_data
        self.trade_history = trade_history
        self.listings = listings
        self.position_limits = position_limits

    def construct_order_depths(group) -> dict[str, OrderDepth]:
        order_depths = {}
        for _, row in group.iterrows():
            product = row['product']
            order_depth = OrderDepth()

            order_depth.buy_orders = {}
            for i in range(1, 4):
                bid_price = row[f'bid_price_{i}']
                bid_volume = row[f'bid_volume_{i}']
                if pd.notnull(bid_price):
                    order_depth.buy_orders[bid_price] = bid_volume

            order_depth.sell_orders = {}
            for i in range(1, 4):
                ask_price = row[f'ask_price_{i}']
                ask_volume = -row[f'ask_volume_{i}']
                if pd.notnull(ask_price):
                    order_depth.sell_orders[ask_price] = ask_volume

            order_depths[product] = order_depth
        return order_depths

    def match_orders(result: dict[str, list[Order]], order_depths: dict[str, OrderDepth]):
        for product, orders in result.items():
            order_depth = order_depths[product]
            buy_orders = sorted(order_depth.buy_orders.items(), reverse=True)
            sell_orders = sorted(order_depth.sell_orders.items())
            


listings = {
    'RAINFOREST_RESIN': Listing(symbol='RAINFOREST_RESIN', product='RAINFOREST_RESIN', denomination='SEASHELLS'),
    'KELP': Listing(symbol='KELP', product='KELP', denomination='SEASHELLS')
}

position_limits = {
    'RAINFOREST_RESIN': 20,
    'KELP': 20
}

trader_data = ''
own_trades = {}
market_trades = {}
position = {}
observation = 0

for timestamp, group in market_data.groupby('timestamp'):
    order_depths = construct_order_depths(group)
    trading_state = TradingState(trader_data, timestamp, listings, order_depths, own_trades, market_trades, position, observation)
    result, conversions, trader_data = trader.run(trading_state)
    print(result)
    break

Timestamp: 0
Market bids: {10002: 1, 9996.0: 2.0, 9995.0: 29.0}
Market asks: {10004: -2, 10005.0: -29.0}
Resin orders: [(RAINFOREST_RESIN, 10002, -1)]
{'RAINFOREST_RESIN': [(RAINFOREST_RESIN, 10002, -1)], 'KELP': []}
